In [1]:
import numpy as np
import scipy.optimize
#import trfp

In [2]:
arr = np.array([1,2,3,4,5,6])
print(np.delete(arr, [0,1,2]))

[4 5 6]


In [3]:
def __multipole(order, skew, strength, x_pos, y_pos):
    """Returns the magnitude of a B-field given multipole parameters
    and position.
    """
    # Takes a multipole strength A normalized to 4.5 cm
    r_pos = np.sqrt(x_pos**2 + y_pos**2)
    theta = np.arctan2(y_pos, x_pos)
    if skew == 0:
        b_magnitude = strength * (r_pos/4.5)**order * np.cos(order*theta)
    if skew == 1:
        b_magnitude = strength * (r_pos/4.5)**order * np.sin(order*theta)
    return b_magnitude

In [4]:
FP6_X = np.array([-3, 0, 3, -3, 0, 3])
FP6_Y = np.array([7.7, 7.7, 7.7, -7.7, -7.7, -7.7])

J_6_PROBE = np.array([[1.0, 0.0, 0.0, 0.0, 2.631605],
                      [0.0, 1.0, 0.0, 0.0, 0.0],
                      [0.0, 0.0, 1.0, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 1.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 1.0]])

J_6_PROBE_OFFSET = np.array([[1.0, 0.222222, 0.0, 0.0, 2.680987],
                             [0.0, 1.0, 0.0, 0.0, 0.444444],
                             [0.0, 0.0, 1.0, 0.444444, 0.0],
                             [0.0, 0.0, 0.0, 1.0, 0.0],
                             [0.0, 0.0, 0.0, 0.0, 1.0]])

J_4_PROBE = np.array([[1.0, 0.0, 0.0, 0.0, 2.927901],
                      [0.0, 1.0, 0.0, 0.0, -0.666667],
                      [0.0, 0.0, 1.0, -0.666667, 0.0],
                      [0.0, 0.0, 0.0, 1.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 1.0]])

J_4_PROBE_ST41 = np.array([[1.0, 0.0, 0.0, 0.0, 2.927901],
                           [0.0, 1.0, 0.0, 0.0, 0.0],
                           [0.0, -0.194805, 1.0, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 1.0, -0.194805],
                           [0.0, 0.0, 0.0, 0.0, 1.0]])

J_4_PROBE_ST37_ST39 = np.array([[1.0, 0.0, 0.0, 0.0, 2.927901],
                                [0.0, 1.0, 0.0, 0.0, 0.0], 
                                [0.0, -0.194805, 1.0, 0.0, 0.0], 
                                [0.0, 0.0, 0.0, 1.0, -0.194805], 
                                [0.0, 0.0, 0.0, 0.0, 1.0]])

In [5]:
THETA_FP_6 = np.array([np.array([1, 1, 1, 1, 1, 1])/6.,  # dipole
                       np.array([1, 0, -1, 1, 0, -1])/-12.*4.5,  # n quad
                       np.array([1, 1, 1, -1, -1, -1])/46.2*4.5,  # s quad
                       np.array([1, 0, -1, -1, 0, 1])/-92.4/2*4.5**2,  # s sext
                       np.array([1, -2, 1, 1, -2, 1])/18./2*4.5**2,  # n sext
                       np.array([1, -2, 1, -1, 2, -1])/-138.6*4.5**3]  # NOT oct, no idea what this is
                     )

THETA_FP_4 = np.array([np.array([1, 0, 1, 0])/2.,  # dipole
                       np.array([1, -1, 1, -1])/-6.*4.5,  # n quad
                       np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                       np.array([1, -1, -1, 1])/-46.2/2*4.5**2]  # sext
                     )

# four probe stations in the garage (stations 37, 39, 41) have a different arrangement of probes
# STATION 37, 39
# TM, TO, BI, BM
THETA_FP_4_ST37_ST39 = np.array([np.array([1, 0, 0, 1])/2.,  #dipole
                                 np.array([-1, 1, -1, 1])/6.*4.5,  # n quad
                                 np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                                 np.array([-1, 1, 1, -1])/46.2/2*4.5**2]  # sext?
                               )
# STATION 41
# TO, TM, BI, BM
THETA_FP_4_ST41 = np.array([np.array([0, 1, 0, 1])/2.,  #dipole
                            np.array([1, -1, -1, 1])/6.*4.5,  # n quad
                            np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                            np.array([1, -1, 1, -1])/46.2/2*4.5**2]  # sext?
                          )

In [6]:
PROBES = [1,1,1,1,1,1] #1 means the probe is working properly, 0, it is dropped or does not exist.
drops = np.array([])
for i in range(len(PROBES)):
    if not PROBES[i]:
        drops = np.append(drops, i)
drops = drops.astype(int)

PROBES = np.delete(PROBES, drops)
FP6_X = np.delete(FP6_X, drops)
FP6_Y = np.delete(FP6_Y, drops)

[]


In [10]:
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0]

_MULTS_FP_N = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X, FP6_Y) for i in range(len(_MULTIPOLE_ORDER) - len(drops))])
_MULTS_FP_N[np.abs(_MULTS_FP_N) < 1.0e-9] = 0

THETA_FPN = np.linalg.pinv(np.transpose(_MULTS_FP_N))
THETA_FPN[np.abs(THETA_FPN) < 1.0e-9] = 0
print(THETA_FPN)

[[ 1.64694444 -2.79388889  1.64694444  1.64694444 -2.79388889  1.64694444]
 [-0.375       0.          0.375      -0.375       0.          0.375     ]
 [ 0.0974026   0.0974026   0.0974026  -0.0974026  -0.0974026  -0.0974026 ]
 [-0.10957792  0.          0.10957792  0.10957792  0.         -0.10957792]
 [ 0.5625     -1.125       0.5625      0.5625     -1.125       0.5625    ]]


In [11]:
print(np.dot(J_6_PROBE, THETA_FP_6[:5]))

[[ 1.64694448 -2.79388896  1.64694448  1.64694448 -2.79388896  1.64694448]
 [-0.375       0.          0.375      -0.375       0.          0.375     ]
 [ 0.0974026   0.0974026   0.0974026  -0.0974026  -0.0974026  -0.0974026 ]
 [-0.10957792  0.          0.10957792  0.10957792  0.         -0.10957792]
 [ 0.5625     -1.125       0.5625      0.5625     -1.125       0.5625    ]]


The above code works for 6 working fp. Below will be testing for 4 probes

In [10]:
#### For some reason this changes when more multipoles are used. I am assuming it has to do with m6 not bing linearly indep.
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 0]
_MULTS_FP6 = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X, FP6_Y) for i in range(len(_MULTIPOLE_ORDER))])
_MULTS_FP6[np.abs(_MULTS_FP6) < 1.0e-9] = 0

THETA_FP6 = np.linalg.pinv(np.transpose(_MULTS_FP6))
THETA_FP6[np.abs(THETA_FP6) < 1.0e-9] = 0
print(THETA_FP6)

[[ 1.64694444 -2.79388889  1.64694444  1.64694444 -2.79388889  1.64694444]
 [-0.00531589  0.          0.00531589 -0.00531589  0.          0.00531589]
 [ 0.0974026   0.0974026   0.0974026  -0.0974026  -0.0974026  -0.0974026 ]
 [-0.10957792  0.          0.10957792  0.10957792  0.         -0.10957792]
 [ 0.5625     -1.125       0.5625      0.5625     -1.125       0.5625    ]
 [ 0.04433057  0.         -0.04433057  0.04433057  0.         -0.04433057]]


In [12]:
PROBES = [0,1,1,0,1,1] #1 means the probe is working properly, 0, it is dropped or does not exist.
drops = np.array([])
for i in range(len(PROBES)):
    if not PROBES[i]:
        drops = np.append(drops, i)
drops = drops.astype(int)

PROBES = np.delete(PROBES, drops)
FP6_X = np.delete(FP6_X, drops)
FP6_Y = np.delete(FP6_Y, drops)

In [30]:
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0]
if len(drops) == 0:
    _MULTS_FP_N = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X, FP6_Y) for i in range(len(_MULTIPOLE_ORDER))])

else:
    _MULTS_FP_N = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X, FP6_Y) for i in range(len(_MULTIPOLE_ORDER) - len(drops) + 1)])


_MULTS_FP_N[np.abs(_MULTS_FP_N) < 1.0e-9] = 0

THETA_FPN = np.linalg.pinv(np.transpose(_MULTS_FP_N))
THETA_FPN[np.abs(THETA_FPN) < 1.0e-9] = 0
THETA_FPN = np.concatenate([THETA_FPN, [[0,0,0,0]]])

In [31]:
print(np.dot(np.linalg.inv(J_4_PROBE)[:4], THETA_FPN))
print(THETA_FP_4)

[[ 0.5         0.          0.5         0.        ]
 [-0.75        0.75       -0.75        0.75      ]
 [ 0.14610382  0.14610397 -0.14610382 -0.14610397]
 [-0.21915584  0.21915584  0.21915584 -0.21915584]]
[[ 0.5         0.          0.5         0.        ]
 [-0.75        0.75       -0.75        0.75      ]
 [ 0.1461039   0.1461039  -0.1461039  -0.1461039 ]
 [-0.21915584  0.21915584  0.21915584 -0.21915584]]


In [ ]:
print(np.dot(J_6_PROBE, THETA_FP_6[:5]))